# Exploring Hackers News Posts

In this project, we'll compare two different types of posts from [Hacker News](https://news.ycombinator.com/), a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

* Do *Ask HN* or *Show HN* receive more comments on average?
* Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction

First, we'll read in the data and remove the headers.

In [1]:
from csv import reader
open_file=open('hacker_news.csv')
read_file=reader(open_file)
hn=list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


## Removing Headers from the Data Set

In [2]:
##Remove the header row
header=hn[0]
hn=hn[1:]

print(header)
print('\n')
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post.

## Extracting Ask HN and Show HN Posts

First, we'll identify posts that begin with either Ask HN or Show HN and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [3]:
ask_posts=[]
show_posts=[]
other_posts=[]

for row in hn:
    title=row[1]
    title=title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
    
print("The number of Ask NH posts is: "+ str(len(ask_posts)))
print("The number of Show NH posts is: "+ str(len(show_posts)))
print("The number of all other posts is: "+ str(len(other_posts)))
print('\n')
print(ask_posts[0])
print('\n')
print(show_posts[0])
print('\n')
print(other_posts[0])

The number of Ask NH posts is: 1744
The number of Show NH posts is: 1162
The number of all other posts is: 17194


['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [4]:
# average ask comments
total_ask_comments=0

for row in ask_posts:
    comment=int(row[4])
    total_ask_comments += comment
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

14.038417431192661


In [5]:
##average show comments
total_show_comments=0

for row in show_posts:
    comment=int(row[4])
    total_show_comments+=comment
    
avg_show_comments=total_show_comments/len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive. *The time in the dataset is EST time.*

In [6]:
import datetime as dt
# create a list of lists with timestamp and number of comments for each post
result_list=[]
for row in ask_posts:
    time_created=row[-1]
    num_comments=int(row[-3])
    result_list.append([time_created,num_comments])
    
print(result_list[:3])
    
    

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1]]


In [7]:
# create dictionaries with number of comments ask posts created at each hour 
# and number of ask posts created during each hour of the day
counts_by_hour={}
comments_by_hour={}

for row in result_list:
    timestamp=row[0]
    timestamp=dt.datetime.strptime(timestamp,"%m/%d/%Y %H:%M")
    hour=timestamp.strftime('%H')
    comment=int(row[1])
    if hour not in counts_by_hour:
        counts_by_hour[hour]=1
        comments_by_hour[hour]=comment
    else:
        counts_by_hour[hour]+=1
        comments_by_hour[hour]+=comment

comments_by_hour

{'00': 447,
 '01': 683,
 '02': 1381,
 '03': 421,
 '04': 337,
 '05': 464,
 '06': 397,
 '07': 267,
 '08': 492,
 '09': 251,
 '10': 793,
 '11': 641,
 '12': 687,
 '13': 1253,
 '14': 1416,
 '15': 4477,
 '16': 1814,
 '17': 1146,
 '18': 1439,
 '19': 1188,
 '20': 1722,
 '21': 1745,
 '22': 479,
 '23': 543}

## Calculating the Average Number of Comments for Ask HN Posts by Hour

In [8]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_comments_per_hour=[]

for hour in comments_by_hour:
    avg=comments_by_hour[hour]/counts_by_hour[hour]
    avg_comments_per_hour.append([hour,avg])

avg_comments_per_hour

[['13', 14.741176470588234],
 ['09', 5.5777777777777775],
 ['19', 10.8],
 ['08', 10.25],
 ['07', 7.852941176470588],
 ['14', 13.233644859813085],
 ['10', 13.440677966101696],
 ['01', 11.383333333333333],
 ['23', 7.985294117647059],
 ['04', 7.170212765957447],
 ['11', 11.051724137931034],
 ['21', 16.009174311926607],
 ['22', 6.746478873239437],
 ['02', 23.810344827586206],
 ['03', 7.796296296296297],
 ['00', 8.127272727272727],
 ['16', 16.796296296296298],
 ['06', 9.022727272727273],
 ['20', 21.525],
 ['12', 9.41095890410959],
 ['05', 10.08695652173913],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['18', 13.20183486238532]]

## Sorting and Printing Values from a List of Lists

In [22]:
swap_avg_comments_per_hour=[]

for row in avg_comments_per_hour:
    hour=row[0]
    avg=row[1]
    swap_avg_comments_per_hour.append([avg,hour])

swap_avg_comments_per_hour
sorted_swap=sorted(swap_avg_comments_per_hour,reverse=True)
print('Top 5 Hours for Ask Posts Comments')
for row in sorted_swap[:5]:
    print(str(row[1]) +':'+'00: '+str(format(row[0],'.2f'))+' average comments per post')

    

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.